## Francisco Teixeira Rocha Aragão 2021031726

## Introdução

O trabalho busca trabalhar com a tarefa de classificação de gêneros de animes baseado na descrição textual (sinopse). O dataset utilizado  (https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset?resource=download&select=anime-dataset-2023.csv) é composto de mais de 24 mil animes, com diferentes colunas descrevendo cada obra, porém no trabalho o foco será na coluna "genre" e "synopsis".

## Metodologia

Foram testados 3 casos diferentes para realizar a classificação:

- Modelo base padrão: O modelo escolhido foi o Deberta Large (https://huggingface.co/MoritzLaurer/deberta-v3-large-zeroshot-v2.0) disponível no Hugging Faces, que é utilizado em tarefas de classificação de texto.

- Modelo treinado com os dados do dataset: O mesmo modelo foi treinado agora para a tarefa específica do trabalho, em que foi realizado o fine-tunning, com apenas a última camada sendo treinada.

- Modelo base padrão + contexto adicional: O mesmo modelo base foi utilizado para classificação. No entanto, o texto passado ao modelo agora contem também informações adicionais sobre cada gênero, com um maior contexto das palavras mais utilizadas em cada caso. Essa tarefa foi feita utilizando a técnica de TF-IDF.

## Resultados

Os resultados, juntamente do código estão disponíveis abaixo. 

- Modelo base: 
    - Acurácia em todas as tags: 32%
    - Acurácia em pelo menos uma tag: 43%

- Modelo treinado:
    - Acurácia em todas as tags: 26%
    - Acurácia em pelo menos uma tag: 36%

- Modelo com contexto adicional: 
    - Acurácia em todas as tags: 36%
    - Acurácia em pelo menos uma tag: 50%

## Conclusão

Percebe-se como, de modo geral, a tarefa de classificação de animes foi muito específica para o modelo, não possuindo bons resultados de modo geral. Desse modo, o modelo base não conseguiu boa performance, acertando menos da metade dos gêneros (mesmo ao considerar como acerto pelo menos um gênero correto). O modelo treinado obteve resultados piores, o que pode ser explicado pela falta de dados para treinamento, além da complexidade de treinar um modelo tão grande. Trabalhos futuros podem focar nessa tarefa, buscando os melhores parâmetros para treinamento, além de alocarem mais recursos para a tarefa. Já o modelo com contexto adicional obteve os melhores resultados. Percebe-se que a adição de contexto adicional foi uma estratégia importante, conseguindo 50% de acurácia ao considerar um acerto quando pelo menos um dos gêneros é classificado corretamente. Desse modo, a estratégia de TF-IDF mostrou-se útil para extrair as palavras mais importantes em cada gênero, sendo um contexto adicional útil para auxiliar a classificação.

In [9]:
import pandas as pd # lidar com dataset
from datasets import Dataset # organizar dados para treino
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline # utilização dos modelos
import json # salvar resultados
from tqdm import tqdm # ver progresso do treinamento
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf para auxiliar classificação dos conteudos
from collections import defaultdict # organização dos dados
import nltk # preprocessamento de texto
from nltk.corpus import stopwords # preprocessamento de texto

nltk.download('stopwords') # stopwords em ingles para serem removidas do texto
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/francisco/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Função para calcular acurácia dos resultados -> acertou todos os generos do anime
def calculate_accuracy_all_genres(results):
    correct = 0
    total = 0

    for _, genres in results.items():
        for _, result in genres.items():
            total += 1
            if result == 'correct':
                correct += 1

    accuracy = correct / total
    print("Accuracy:", accuracy)

# Função para calcular acurácia dos resultados -> acertou pelo menos um genero do anime
def calculate_accuracy_at_least_one_genre(results):
    correct = 0
    total = 0
    for _, genres in results.items():
        total += 1
        if "correct" in genres.values():
            correct += 1

    accuracy = correct / total
    print("Accuracy (at least one correct genre):", accuracy)

## Carregando, visualizando e filtrando dados


In [11]:
# carregando dataset
data = pd.read_csv('./data/anime-dataset-2023.csv')


In [13]:
# vendo nome das colunas
print(data.columns)

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')


In [14]:
data.head(5)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [16]:
# Utilizando apenas as colunas que serão importantes para a classificação 
data = data[['anime_id', 'Name', 'Genres', 'Synopsis']]

data.head()

,anime_id,Name,Genres,Synopsis
0,1,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o..."
2,6,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...


In [17]:
# preprocessamento da coluna de generos dos animes, removendo espaços, separando informações e deixando minusculo

data['Genres'] = data['Genres'].apply(lambda x: x.replace(' ', '').lower())
data['Genres'] = data['Genres'].str.split(', ')

In [18]:
# preprocessamento do campo de sinopse, deixando minusculo e removendo stopwords -> deixando apenas palaras mais importantes e menos comuns

data['Synopsis'] = data['Synopsis'].str.lower()

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# remove stopwords in synopsis
data['Synopsis'] = data['Synopsis'].str.lower().apply(remove_stopwords)

In [19]:
# vendo novo dataset preprocessado

data.head(10)

,anime_id,Name,Genres,Synopsis
0,1,Cowboy Bebop,"[action,awardwinning,sci-fi]","crime timeless. year 2071, humanity expanded a..."
1,5,Cowboy Bebop: Tengoku no Tobira,"[action,sci-fi]","another day, another bounty—such life often un..."
2,6,Trigun,"[action,adventure,sci-fi]","vash stampede man $$60,000,000,000 bounty head..."
3,7,Witch Hunter Robin,"[action,drama,mystery,supernatural]",robin sena powerful craft user drafted stnj—a ...
4,8,Bouken Ou Beet,"[adventure,fantasy,supernatural]","dark century people suffering rule devil, vand..."
5,15,Eyeshield 21,[sports],"shy, reserved, small-statured, deimon high sch..."
6,16,Hachimitsu to Clover,"[comedy,drama,romance]","yuuta takemoto, sophomore arts college, shares..."
7,17,Hungry Heart: Wild Striker,"[comedy,sliceoflife,sports]",younger brother japanese soccer star seisuke k...
8,18,Initial D Fourth Stage,"[action,drama]",takumi fujiwara finally joins ryousuke keisuke...
9,19,Monster,"[drama,mystery,suspense]","dr. kenzou tenma, elite neurosurgeon recently ..."


In [20]:
# separando dataset em treino e teste
# 80% para treino e 20% para teste

train = data.sample(frac=0.8, random_state=0)
test = data.drop(train.index)

In [21]:
data.shape, train.shape, test.shape

((24905, 4), (19924, 4), (4981, 4))

In [40]:
# organizando os dados para treino -> salvando lista de generos


# pegando todos os generos possiveis
all_genres = list(data['Genres'].explode().unique())

print(all_genres)

genres = set() # organizando dados em um set para evitar repetições
for i in all_genres:
    genres_separated = i.replace(' ', '').split(',')
    for j in genres_separated:
        genres.add(j)


genres = list(genres) # salvando dados em uma lista pra facilitar o trabalho
print(genres)

['action,awardwinning,sci-fi', 'action,sci-fi', 'action,adventure,sci-fi', 'action,drama,mystery,supernatural', 'adventure,fantasy,supernatural', 'sports', 'comedy,drama,romance', 'comedy,sliceoflife,sports', 'action,drama', 'drama,mystery,suspense', 'action,adventure,fantasy', 'action,sports', 'comedy,romance', 'action,adventure,comedy,sci-fi,ecchi', 'action,drama,sci-fi', 'action,supernatural', 'comedy,gourmet', 'action,avantgarde,awardwinning,drama,sci-fi,suspense', 'drama,sci-fi', 'avantgarde,drama,sci-fi', 'action,adventure,drama,fantasy,horror', 'action,awardwinning,mystery,sci-fi,suspense', 'action,drama,romance', 'action,adventure,comedy,romance', 'drama', 'action,adventure,horror,sci-fi,supernatural', 'adventure,fantasy,mystery', 'comedy,romance,supernatural', 'action,drama,romance,supernatural', 'awardwinning,comedy,drama,sci-fi,sports', 'action,adventure,fantasy,horror,sci-fi', 'adventure,fantasy,sci-fi', 'comedy,drama,sliceoflife', 'adventure,drama,horror,romance,sci-fi', '

In [41]:
print(len(genres))

22


In [27]:
# vendo dados que serão utilizados
anime_synopsis = test['Synopsis'].iloc[0]
anime_genre = test['Genres'].iloc[0]
print(anime_synopsis)
print("---")
print(anime_genre)

moments prior naruto uzumaki's birth, huge demon known kyuubi, nine-tailed fox, attacked konohagakure, hidden leaf village, wreaked havoc. order put end kyuubi's rampage, leader village, fourth hokage, sacrificed life sealed monstrous beast inside newborn naruto. now, naruto hyperactive knuckle-headed ninja still living konohagakure. shunned kyuubi inside him, naruto struggles find place village, burning desire become hokage konohagakure leads great new friends, also deadly foes.
---
['action,adventure,fantasy']


## Teste 1: Início das classificações -> Modelo base

In [28]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")  # change the model identifier here


Device set to use cpu


In [ ]:
# realizando tarefa de predição dos gêneros dos animes

results = {}

# iterando sobre o dataset, coletando informações e construindo texto para ser classificado
for index, row in tqdm(test.iterrows(), total=test.shape[0], desc="Processing anime"):
    anime_id = row['anime_id']
    anime_synopsis = row['Synopsis']
    anime_name = row['Name']
    anime_genre = row['Genres']

    text = 'The anime is called ' + anime_name + '. The synopsis of the anime is: ' + anime_synopsis # texto para ser classificado = nome do anime + sinopse
    output = zeroshot_classifier(text, list(genres), multi_label=True) # realizando classificação 

    # organziando os resultados
    anime_genre_len = len(anime_genre.split(','))
    top_k_labels_predicted = output['labels'][0:anime_genre_len]

    results[anime_id] = {}

    for i in range(anime_genre_len): # verificando se resultados foram corretos
        genre = anime_genre.split(',')[i].strip()
        results[anime_id][genre] = "error"
        if genre in top_k_labels_predicted: 
            results[anime_id][genre] = "correct"

# salvando resultados em um json -> modelo base
with open('results_based_model.json', 'w') as f:
    json.dump(results, f, indent=4)

In [6]:
# abrindo resultados e calculando acurácia
with open('results_based_model.json') as f:
    results_baseline = json.load(f)

calculate_accuracy_all_genres(results_baseline)

calculate_accuracy_at_least_one_genre(results_baseline)

Accuracy: 0.320385012031626
Accuracy (at least one correct genre): 0.4364585424613531


## Teste 2: Iniciando processo de treinamento -> Fine-tuning

In [43]:
# organizando generos dos animes
unique_genres = sorted(genres)

print("Unique genres:", unique_genres)

Unique genres: ['action', 'adventure', 'avantgarde', 'awardwinning', 'boyslove', 'comedy', 'drama', 'ecchi', 'erotica', 'fantasy', 'girlslove', 'gourmet', 'hentai', 'horror', 'mystery', 'romance', 'sci-fi', 'sliceoflife', 'sports', 'supernatural', 'suspense', 'unknown']


In [44]:

# Função pra preprocessar o dataset e organizar os dados no formato correto para treino -> novo nome das colunas (text e labels)
def preprocess(df):
    df['text'] = 'The synopsis of the anime is: ' + df['Synopsis']
    
    # Encode labels as binary vectors
    genre_to_id = {genre: idx for idx, genre in enumerate(unique_genres)}
    
    def encode_labels(genres):
        labels = [0] * len(unique_genres)
        if isinstance(genres, str): # checando se é string
            for genre in genres.split(','):
                labels[genre_to_id[genre.strip()]] = 1
        return labels
    
    df['labels'] = df['Genres'].apply(encode_labels)
    return df

In [ ]:
# organização dos dados antes de iniciar o treinamento 


# atualizando datasets de treino e teste
train = preprocess(train)
test = preprocess(test)

# convertendo datasets para o formato do Hugging Face
train_dataset = Dataset.from_pandas(train[['text', 'labels']])
test_dataset = Dataset.from_pandas(test[['text', 'labels']])

# carregando tokenizador
model_name = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)


# função para tokenizar os dados
def tokenize(batch):
    tokenized = tokenizer(batch['text'], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = batch["labels"]
    return tokenized

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:

# carregando modelo
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_genres),  # Set to the number of unique genres (42)
    ignore_mismatched_sizes=True    # Ignore size mismatches
)

# congelando os pesos do modelo base -> apenas a última camada será treinada, muito difícil treinar um modelo do zero
for param in model.base_model.parameters():
    param.requires_grad = False

# definindo parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    disable_tqdm=False, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# realizando treinamento
trainer.train()

In [ ]:
# Salvando modelo treinado
model.save_pretrained('./model/fine-tuned-anime-genre-model')
tokenizer.save_pretrained('./model/fine-tuned-anime-genre-model')

In [ ]:
# Carregando modelo treinado
model_path = './model/fine-tuned-anime-genre-model'
zeroshot_classifier = pipeline("zero-shot-classification", model=model_path)

In [ ]:
# realizando predição com modelo treinado
results = {}

for index, row in tqdm(test.iterrows(), total=test.shape[0], desc="Processing anime"):
    anime_id = row['anime_id']
    anime_synopsis = row['Synopsis']
    anime_name = row['Name']
    anime_genre = row['Genres']

    # preparando texto para classificação
    text = f"The anime is called {anime_name}. The synopsis of the anime is: {anime_synopsis}"

    # realizando prediçao
    output = zeroshot_classifier(text, unique_genres, multi_label=True)

    anime_genre_len = len(anime_genre.split(','))
    top_k_labels_predicted = output['labels'][0:anime_genre_len]

    # salvando resultados
    results[anime_id] = {}
    for i in range(anime_genre_len):
        genre = anime_genre.split(',')[i].lower()
        results[anime_id][genre] = "error"
        if genre in top_k_labels_predicted:
            results[anime_id][genre] = "correct"

# salvando resultados em arquivo json
with open('results_finetuned_model.json', 'w') as f:
    json.dump(results, f, indent=4)

In [45]:
# abrindo resultados e calculando acurácia
with open('results_finetuned_model.json') as f:
    results_baseline = json.load(f)

calculate_accuracy_all_genres(results_baseline)

calculate_accuracy_at_least_one_genre(results_baseline)

Accuracy: 0.26045605591841414
Accuracy (at least one correct genre): 0.3631800843204176


## Teste 3: Zero shot -> Adicionando mais contexto no prompt de classificação usando TF-IDF

In [ ]:
# Organiznado dados para realizar tf-idf e encontrar palavras importantes para cada gênero
genre_synopsis = defaultdict(str)

for index, row in data.iterrows():
    synopsis = row['Synopsis'].lower()
    for genre in row['Genres']:
        genre = genre.split(',')
        for i in genre:
            genre_synopsis[i] += " " + synopsis

# iniciando tf-idf
tfidf = TfidfVectorizer(stop_words='english')  # utilizando stopwords em ingles para serem removidas

# organizando os dados para realizar tf-idf
genre_texts = list(genre_synopsis.values())
genre_names = list(genre_synopsis.keys())

# realizando tf-idf
tfidf_matrix = tfidf.fit_transform(genre_texts)
vocab = tfidf.get_feature_names_out()


tfidf_array = tfidf_matrix.toarray()
word_genre_scores = tfidf_array.T  # Transpondo a matriz para pegar relação palavra-genero. Linha = palavra, colunas = generos

# salvando palavras unicas pra cada genero
unique_words_per_genre = defaultdict(list)

# organizando palavras unicas para cada genero
for word_idx, word in enumerate(vocab):
    word_scores = word_genre_scores[word_idx]
    # pegando palavras com score > 0 -> palavras que são unicas para um genero
    non_zero_genres = sum(score > 0 for score in word_scores)
    if non_zero_genres == 1:  
        genre_idx = word_scores.argmax()  
        unique_words_per_genre[genre_names[genre_idx]].append((word, word_scores[genre_idx]))

# imprimindo resultados do tf-idf
for genre, words in unique_words_per_genre.items():
    print(f"Genre: {genre}")
    print("Unique Words:", [word for word, score in sorted(words, key=lambda x: x[1], reverse=True)])
    print()

# salvando resultados em um dicionario [genero: [palavras unicas]]
unique_words_per_genre_dict = {}
# salvando as 10 palavras mais importantes para cada genero
for genre, words in unique_words_per_genre.items():
    unique_words_per_genre_dict[genre] = [word for word, _ in sorted(words, key=lambda x: x[1], reverse=True)][:10]

# imprimindo resultados
print(unique_words_per_genre_dict)

Genre: comedy
Unique Words: ['coffy', 'tatsu', 'hetalia', 'meisaku', 'umaru', 'sextuplets', 'takatoshi', 'ashuai', 'kongming', 'lishi', 'ruguo', 'yiqun', 'agetarou', 'hachiken', 'heine', 'kaishain', 'makibao', 'cofun', 'coro', 'ousai', 'rakko', 'tsuda', 'ub', 'dokutake', 'ketsuekigata', 'mak', 'minoa', 'otowakan', 'reinya', 'tumulus', 'wansa', 'absurdist', 'akuei', 'appare', 'babyshiba', 'binan', 'broomstown', 'chio', 'elfs', 'evilrun', 'farts', 'grannzreich', 'haecceitas', 'hagemaru', 'hifumi', 'himote', 'househusband', 'kojimachi', 'kyattsuyama', 'mameko', 'mitchiri', 'musik', 'nameko', 'nma', 'ogiue', 'ohanami', 'okehazama', 'onosaka', 'pacusi', 'plasmagica', 'poli', 'potemayo', 'puns', 'shibainuko', 'shuukan', '419', 'abciee', 'bald', 'brooms', 'celeb', 'chatter', 'chocotan', 'churuya', 'cuddly', 'daikon', 'dochamon', 'fuuta', 'hametsu', 'hanta', 'hennako', 'inuwari', 'itoshiki', 'iyami', 'jantama', 'jerry', 'juzu', 'kaatan', 'kakko', 'kakusen', 'katsuhito', 'kine', 'koke', 'konata

In [ ]:

# carregando modelo
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")  # change the model identifier here

# função para adicionar contexto no prompt baseado nas palavras achadas pelo tf-idf pra cada genero
# "enriquecimento de prompt" -> adiciono informação extra para ajudar o modelo a classificar melhor
def add_context_based_on_keywords(synopsis, genre_keywords):
    context = ""
    for genre, keywords in genre_keywords.items():
        if any(keyword.lower() in synopsis.lower() for keyword in keywords):
            # novo texto a ser adicionado
            context += f" This anime involves {genre.lower()} elements, including {', '.join(keywords)}."

    return context

results = {}

for index, row in tqdm(test.iterrows(), total=test.shape[0], desc="Processing anime"):
    anime_id = row['anime_id']
    anime_synopsis = row['Synopsis']
    anime_name = row['Name']
    anime_genre = row['Genres']

    # pegando novo texto a ser adicionado
    augmented_text = add_context_based_on_keywords(anime_synopsis, unique_words_per_genre_dict)

    text = 'The anime is called ' + anime_name + '. The synopsis of the anime is: ' + anime_synopsis + ". " + augmented_text

    # predição
    output = zeroshot_classifier(text, unique_genres, multi_label=True)

    # organizando resultados
    anime_genre_len = len(anime_genre[0].split(','))
    top_k_labels_predicted = output['labels'][0:anime_genre_len]

    results[anime_id] = {}
    for i in range(anime_genre_len):
        genre = anime_genre[0].split(',')[i].lower()
        results[anime_id][genre] = "error"
        if genre in top_k_labels_predicted:
            results[anime_id][genre] = "correct"

# salvando resultados em um json
with open('results_context_classification.json', 'w') as f:
    json.dump(results, f, indent=4)

In [49]:
# compare the accuracy with the previous model
with open('results_context_classification.json') as f:
    results_context = json.load(f)

calculate_accuracy_all_genres(results_context)

calculate_accuracy_at_least_one_genre(results_context)

Accuracy: 0.3667927122722585
Accuracy (at least one correct genre): 0.5035133507327846
